In [18]:
import os
import numpy as np
from numpy import array
from numpy import argmax
import imageio
import matplotlib.pyplot as plt
import pandas as pd
import glob
import keras
import time
from keras import backend as K
from keras.layers.core import Dense
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import CSVLogger, EarlyStopping, TensorBoard, ModelCheckpoint 
import matplotlib.pyplot as plt
import cv2
from scipy.misc import imread, imresize
from keras.applications.vgg16 import VGG16
from keras_vggface.vggface import VGGFace
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras import backend as K
from keras.utils import to_categorical
from numpy import genfromtxt

from sklearn.model_selection import train_test_split

%matplotlib inline

Using TensorFlow backend.


In [2]:
#Read Data
my_data = pd.read_csv('train_data.csv', header=None)

In [23]:
#Helper to show image
def plotImage(image):
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(image)
    axarr[0].grid()
    axarr[0].set_title('Image')

In [5]:
#Create np array of the images, reshape to (48,48) then convert to 3D
images = []

for index, row in my_data.iterrows():
    image = np.reshape(row, (48,48))
    image = imresize(image, (224,224,3))
    image = np.reshape(image, image.shape + (1,))
    images.append(image)
    

C:\Users\user\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  import sys


In [6]:
#Convert to NP array
X_train = np.asarray(images)

In [7]:
#Read training data
targets = pd.read_csv('train_target.csv', header = None)

In [8]:
#Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_train, targets, test_size=0.10, random_state=3)

In [9]:
#Apply padding to images to feed into network.
pad = ((0,0),)*3 + ((0,2),)
X_train = np.pad(X_train, pad, 'constant', constant_values = 0)
X_test = np.pad(X_test, pad, 'constant', constant_values = 0)

In [10]:
print(X_train.shape)

(14557, 224, 224, 3)


In [11]:
#Convert to One-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [49]:
#Create the VGG Face model
vgg_face_model = VGGFace(model = 'vgg16', include_top = False, weights='vggface', input_shape=(224,224,3))
    
vgg_face_model.summary()

#Freeze all layers
for layer in vgg_face_model.layers[:-1]:
    layer.trainable = False

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [56]:
#Add the fully connected layers and classifier
from keras import regularizers
LL = vgg_face_model.get_layer('pool5').output
x = Flatten(name='flatten')(LL)
x = Dense(64,name = 'fc7')(x)
x = Dropout(0.5)(x)
out = Dense(3, activation='softmax',name='classifier')(x)
custom_vgg_face_model = Model(vgg_face_model.input, out)

In [57]:
#Specify optimizer, loss function, and metrics to track
sgd = SGD(lr=0.0003, decay=1e-5, momentum=0.9)
custom_vgg_face_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [58]:
# fine-tune the model
filepath="vgg_face_gender_weights_improvment-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callback_list = [checkpoint]
custom_vgg_face_model.fit(x=X_train,y=y_train, batch_size=32, epochs=75, verbose=1, callbacks=callback_list, validation_data=(X_test, y_test))

Train on 14557 samples, validate on 1618 samples
Epoch 1/75
14557/14557 [==============================] - 107s 7ms/step - loss: 5.2842 - acc: 0.6414 - val_loss: 4.2213 - val_acc: 0.7169

Epoch 00001: val_acc improved from -inf to 0.71693, saving model to vgg_face_gender_weights_improvment-01-0.72.hdf5
Epoch 2/75
14557/14557 [==============================] - 100s 7ms/step - loss: 4.1419 - acc: 0.7220 - val_loss: 3.8645 - val_acc: 0.7379

Epoch 00002: val_acc improved from 0.71693 to 0.73795, saving model to vgg_face_gender_weights_improvment-02-0.74.hdf5
Epoch 3/75
14557/14557 [==============================] - 100s 7ms/step - loss: 3.7084 - acc: 0.7503 - val_loss: 3.3536 - val_acc: 0.7756

Epoch 00003: val_acc improved from 0.73795 to 0.77565, saving model to vgg_face_gender_weights_improvment-03-0.78.hdf5
Epoch 4/75
14557/14557 [==============================] - 100s 7ms/step - loss: 3.5047 - acc: 0.7670 - val_loss: 3.5528 - val_acc: 0.7528

Epoch 00004: val_acc did not improve from

KeyboardInterrupt: 

In [59]:
#load weights if needed
custom_vgg_face_model.load_weights("vgg_face_gender_weights_improvment-06-0.80.hdf5")
print("Loaded model from disk")

Loaded model from disk


In [19]:
#read file names from sample_submissions
test_data = pd.read_csv('test_data.csv', header=None)

In [20]:
#Create np array of testing images
test_images = []

for index, row in test_data.iterrows():
    im = np.reshape(row, (48,48))
    im = imresize(im, (224,224,3))
    im = np.reshape(im, im.shape + (1,))
    test_images.append(im)
    

C:\Users\user\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  import sys


In [21]:
#convert to np array
test_images2 = np.asarray(test_images)

In [25]:
#Apply padding to test images for predictions
pad = ((0,0),)*3 + ((0,2),)
test_images2 = np.pad(test_images2, pad, 'constant', constant_values = 0)

In [153]:
#Generate predictions
predictions = custom_vgg_face_model.predict(test_images2)

In [154]:
#Get argmax of predictions
arg_predictions = np.argmax(predictions, axis=1)

In [155]:
#Create df of predictions
emotion = pd.DataFrame(arg_predictions) 

In [156]:
#read file names from sample_submissions
submission = pd.read_csv('sample-submission.csv')

In [158]:
#Create final submission
result = pd.concat([submission, emotion],axis=1)
result.columns = ['Id', 'C1', 'Category']
result = result.drop(columns=['C1'], inplace=True)

In [160]:
#Save results to CSV
result.to_csv('answers_op3_vgg_face_2.csv')

# Method 2: SVM w/ Landmarks

In [61]:
import cv2
import glob
import random
import math
import numpy as np
import dlib
import itertools
import os
import pandas as pd
from sklearn.svm import SVC
from sklearn import grid_search

C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [78]:
#Specify classes, initialize DLIB detector and predictor
emotions = ["0", "1", "2"] 
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#Create SVM model
clf = SVC(cache_size=200, C=1, gamma = 0.001, class_weight=None,  kernel='linear',
  probability=False, random_state=None, 
  tol=0.001, verbose=True)



# Data Preprocessing

In [4]:
#Read data, and concatenate the dataframes
my_data = pd.read_csv('train_data.csv', header=None)
labels = pd.read_csv('train_target.csv', header=None)
labels.rename(columns={labels.columns[0]:'Target'}, inplace=True)

dataframe = pd.concat([my_data, labels],axis=1)

In [235]:
#Split training and testing 80% and 20% respectively
#Will split into classification directories and save images into a folder.
data_size = len(dataframe.index)
train_size = dataframe - (data_size * 0.2)
count = 0

for index, row in dataframe.iterrows():
    image = np.reshape(row[:-1], (48,48))
    target = str(row['Target'])
    str_index = str(index)

    if count<train_size:
        
        if os.path.isdir('data/train/' + target):
            cv2.imwrite('data/train/' + target + '/' + str_index + '.jpg', image)
        else:
            os.makedirs('data/train/'+ target)
            cv2.imwrite('data/train/' + target + '/' +  str_index + '.jpg', image)
            
        count = count + 1
    
    #For the last 20% of data, copy to validation with same logic as previous.
    else:
        if os.path.isdir('data/validation/' + target):
            cv2.imwrite('data/validation/' + target + '/' + str_index + '.jpg', image)
        else:
            os.makedirs('data/validation/'+ target)
            cv2.imwrite('data/validation/' + target + '/' + str_index + '.jpg', image)
            
        count = count + 1
    

C:\Users\user\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [4]:
#Read testing Data
testdf = pd.read_csv('test_data.csv', header=None)

In [13]:
#Create folder of images for testing data
for index, row in testdf.iterrows():
    image = np.reshape(row, (48,48))
    str_index = str(index)
    
    cv2.imwrite('data/kaggle_test/' + str(index) + ".jpg", image)

C:\Users\user\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [ ]:
#Save all images to one folder
for index, row in my_data.iterrows():
    image = np.reshape(row[:-1], (48,48))
    target = str(row['Target'])
    str_index = str(index)
    
    cv2.imwrite('data/all/' + target + "/" + str_index + ".jpg", image)

# Helper Functions

#### These functions were built with reference from http://www.paulvangent.com/2016/08/05/emotion-recognition-using-facial-landmarks/

In [50]:
#Function retrieves files using file globbing
def get_files(emotion): 
    files = glob.glob("data\\all\\%s\\*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] 
    validation = files[-int(len(files)*0.2):]
    return training, validation

In [51]:
#Function retrieves test files using file globbing
def get_test_files():
    files = glob.glob("data\\kaggle_test\\**")
    test_data = files[:]
    return test_data

In [ ]:
#Initialize data dictionary to store landmarks
data = {} 

#Function utilizes dlib to create landmarks, vectorize them, and store them into the dictionary.  (See url for explanation of arithmetic)
def get_landmarks(image):
    detections = detector(image, 1)
    for k,d in enumerate(detections): #For all detected face instances individually
        shape = predictor(image, d) #Draw Facial Landmarks with the predictor class
        xlist = []
        ylist = []
        #for i in range(1,68): #Store X and Y coordinates in two lists
            #xlist.append(float(shape.part(i).x))
            #ylist.append(float(shape.part(i).y))
        for p in shape.parts():
            xlist.append(float(p.x))
            ylist.append(float(p.y))
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        landmarks_vectorized = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorized.append(w)
            landmarks_vectorized.append(z)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            landmarks_vectorized.append(dist)
            landmarks_vectorized.append((math.atan2(y, x)*360)/(2*math.pi))
        data['landmarks_vectorized'] = landmarks_vectorized
    if len(detections) < 1:
        data['landmarks_vestorized'] = "error"

In [91]:
#Function returns training and testing dataset and labels along with vectorized landmarks
def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        print(" Working on %s" %emotion)
        training, prediction = get_files(emotion)
        for item in training:
            image = cv2.imread(item) 
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
            gray = cv2.resize(gray, (128,128))
            clahe_image = clahe.apply(gray)
            get_landmarks(clahe_image)
            if data['landmarks_vectorized'] == "error":
                print("no face detected on this one")
            else:
                training_data.append(data['landmarks_vectorized'])
                training_labels.append(emotions.index(emotion))
        for item in prediction:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = cv2.resize(gray, (128,128))
            clahe_image = clahe.apply(gray) 
            get_landmarks(clahe_image)
            if data['landmarks_vectorized'] == "error":
                print("No Face")
            else:
                prediction_data.append(data['landmarks_vectorized'])
                prediction_labels.append(emotions.index(emotion))
    return training_data, training_labels, prediction_data, prediction_labels

In [54]:
#Function builts kaggle_test set and creates landmarks to pass into SVM
def make_test_set():
    test_data = []
    t_data = get_test_files()
    
    for item in t_data:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
            clahe_image = clahe.apply(gray)
            get_landmarks(clahe_image)
            if data['landmarks_vectorized'] == "error":
                print("No Face")
            else:
                test_data.append(data['landmarks_vectorized']) 
                
    return test_data
        
    

# Begin Training

In [60]:
from sklearn import metrics
import pickle
from sklearn.preprocessing import MinMaxScaler

In [93]:
accur_lin = []
num_iters = 2

#Create dataset, train the model.
for i in range(0,num_iters):
    print("Generating datasets %s" %i) 
    training_data, training_labels, prediction_data, prediction_labels = make_sets()
    npar_train = np.array(training_data) 
    npar_trainlabs = np.array(training_labels)
    print("Training SVM Linear #%s" %i) 

    scaling = MinMaxScaler(feature_range=(0,1)).fit(npar_train)
    npar_train = scaling.transform(npar_train)
    clf.fit(npar_train, training_labels)
    pickle.dump(clf, open('SVM_' + str(i) + '.sav', 'wb'))
    print("Obtaining accuracies Trial: %s" %i) 
    npar_pred = np.array(prediction_data)
    npar_pred = scaling.transform(npar_pred)
    pred_lin = clf.score(npar_pred, prediction_labels)
    print("linear: ", pred_lin)
    accur_lin.append(pred_lin)
print("Mean value lin svm: %s" %np.mean(accur_lin)) 

Generating datasets 0
 Working on 0
 Working on 1
 Working on 2
Training SVM Linear #0
[LibSVM]Obtaining accuracies Trial: 0
linear:  0.7771251931993818
Generating datasets 1
 Working on 0


KeyboardInterrupt: 

In [39]:
#get testing_data
testing_data = make_test_set()

In [40]:
#Convert to np array
tt_data = np.array(testing_data)

In [41]:
#Scale the data before passing into SVM
scaling = MinMaxScaler(feature_range=(0,1)).fit(tt_data)
predictions_scaled = scaling.transform(tt_data)

In [42]:
#Generate predictions
predictions = clf.predict(predictions_scaled)

In [45]:
from scipy import stats
stats.describe(predictions)

DescribeResult(nobs=3965, minmax=(0, 2), mean=0.9520807061790668, variance=0.5572389844031338, skewness=0.07762634686324128, kurtosis=-1.2023994143689478)

In [31]:
#Generate DF of predictions
resultsdf = pd.DataFrame(predictions)

In [35]:
#read file names from sample_submissions, create final DF
submission2 = pd.read_csv('sample-submission.csv')
result = pd.concat([submission2, resultsdf],axis=1)
result.columns = ['Id', 'C1', 'Category']
result.drop(columns=['C1'], inplace=True)

In [56]:
training_data, training_labels, prediction_data, prediction_labels = make_sets()

 Working on 0
 Working on 1
 Working on 2
